In [0]:
import spacy

In [0]:
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
file_name = 'story.txt'

with open(file_name, 'w') as file:
    pass

print(f"Created file: {file_name}")

Created file: story.txt


In [0]:
!pip install openai==1.13.3

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2478635f-02a8-4e35-b883-57295f4afd5b
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Not uninstalling openai at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/en

In [0]:
dbutils.library.restartPython()

In [0]:
import asyncio
from openai import AsyncAzureOpenAI

In [0]:
def find_part_of_speech(story):
    tokens_spacy = nlp(story)
    pos_tags = [(token.text, token.pos_) for token in tokens_spacy]
    Part_of_speech_dict = {}
    for token, pos in pos_tags:
        if pos not in Part_of_speech_dict:
            Part_of_speech_dict[pos] = []
        Part_of_speech_dict[pos].append(token)
        
    print("Find all Part-of-speeches in the generated story")
    for i in Part_of_speech_dict:
        print(i,": ",Part_of_speech_dict[i])

In [0]:
async def main():
  try:
        # Get configuration settings
        # Configuration settings
        azure_oai_endpoint =""
        azure_oai_key =""
        azure_oai_deployment ="mygptmodel"

        # Configure the Azure OpenAI client
        client = AsyncAzureOpenAI(
            azure_endpoint = azure_oai_endpoint,
            api_key=azure_oai_key,
            api_version="2024-02-15-preview"
        )

        while True:
            print('Story Generator')
            print('Type quit to finish')
            user_input = input('Enter the Theme of a story you want to Generate:')

            if user_input.lower() == 'quit':
                print('Exiting program...')
                break

            file = open(file="function.py", encoding="utf8").read()

            prompt = user_input + file
            await call_openai_model(prompt, model=azure_oai_deployment, client=client)

  except Exception as ex:
        print(ex)

In [0]:
async def call_openai_model(prompt, model, client):
    # Provide a basic user message, and use the prompt content as the user message
    system_message = "You are an expert in creating 100 word complete stories out of any theme given to you"
    user_message = prompt

    # Format and send the request to the model
    messages =[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    # Call the Azure OpenAI model
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.9,
        max_tokens=1000
    )

    # Print the response to the console
    print("A story of 100 words about the prompt ")
    print(response.choices[0].message.content)

    # Write the response to a file
    results_file = open(file="story.txt", mode="w", encoding="utf8")
    results_file.write(response.choices[0].message.content)
    print("\nResponse written to result/story.txt\n\n")

    find_part_of_speech(response.choices[0].message.content)
     
     


In [0]:
await main()

Story Generator
Type quit to finish


Enter the Theme of a story you want to Generate: Honesty

A story of 100 words about the prompt 
Once upon a time, there was a young mathematician named Alice. She believed that honesty was the key to success in her field. One day, her professor gave her a challenging task - to create a 100-word story about honesty. Alice pondered for a moment and then began to write.

"In the world of equations and numbers, honesty was the guiding principle. Just like the factorial function, honesty multiplied the value of trust. It could not be defined for negative actions, only for positive ones. Honesty, like factorial, started with the smallest unit - truth. And with each honest choice, the value grew exponentially. Alice understood that her success as a mathematician depended on her honesty. And so, she committed to always choosing truth over deception, knowing that it would lead her to infinite possibilities."

Alice's story captured the essence of honesty in a concise and meaningful way, impressing her professor with her perspective.

Response written

Enter the Theme of a story you want to Generate: quit

Exiting program...
